In [4]:
for index,i in enumerate(['sd','sds']):
    print(inf,i)

0 sd
1 sds


In [5]:
from scipy.spatial.distance import jensenshannon

In [7]:
import numpy as np

In [11]:
np.random.randn(2, 4)

array([[ 0.3884779 , -0.10282707, -0.28046676,  0.94322203],
       [ 1.47525405, -0.15213832,  1.22995207, -0.27261961]])

In [67]:
gaussian_filter1d(np.concatenate((np.random.randn(2, 4),np.array([0.0,0,0,0]).reshape(1,4))),sigma=0.5,axis=1)

array([[ 1.01386277,  0.83485232,  2.52871904,  1.31538215],
       [-0.63015232,  0.70128957,  1.51651908,  1.40975201],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [68]:
pred_profile = gaussian_filter1d(np.concatenate((np.random.randn(2, 4),np.array([0.0,0,0,0]).reshape(1,4))),sigma=0.5,axis=1)

In [59]:
pred_profile.shape[0]

2000

In [76]:
pred_profile_prob = pred_profile / np.sum(pred_profile,axis=1,keepdims=True)

/users/vir/anaconda3/envs/basepairmodels_latest/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [77]:
pred_profile_prob

array([[ 0.30717385,  0.01728854,  0.17724785,  0.49828975],
       [-0.11616329,  0.54155731,  0.12448865,  0.45011733],
       [        nan,         nan,         nan,         nan]])

In [66]:
pred_profile/(np.sum(pred_profile,axis=1).reshape(50,1))

array([[-0.1171097 , -0.05630112, -0.21224799, ..., -0.00612443,
         0.02636863, -0.0390545 ],
       [-0.07843144, -0.01195483,  0.00786744, ..., -0.01040716,
         0.00333097,  0.05838523],
       [ 0.01257375, -0.011515  , -0.0609394 , ..., -0.05579533,
        -0.08907486,  0.01459756],
       ...,
       [ 0.09249657, -0.04259359,  0.0238692 , ...,  0.06946889,
        -0.11803326,  0.00277448],
       [-0.01022243,  0.00761536,  0.01068215, ...,  0.01532469,
        -0.00048894,  0.00666527],
       [-0.09880965, -0.02457329,  0.08846037, ...,  0.05089155,
        -0.10591967,  0.00091952]])

In [50]:
np.sum(pred_profile,axis=1)

array([-0.11202276, -1.56859551,  0.        ])

In [38]:
from scipy.ndimage import gaussian_filter1d
??gaussian_filter1d

Signature:
gaussian_filter1d(
    input,
    sigma,
    axis=-1,
    order=0,
    output=None,
    mode='reflect',
    cval=0.0,
    truncate=4.0,
)
Docstring:
1-D Gaussian filter.

Parameters
----------
input : array_like
    The input array.
sigma : scalar
    standard deviation for Gaussian kernel
axis : int, optional
    The axis of `input` along which to calculate. Default is -1.
order : int, optional
    An order of 0 corresponds to convolution with a Gaussian
    kernel. A positive order corresponds to convolution with
    that derivative of a Gaussian.
output : array or dtype, optional
    The array in which to place the output, or the dtype of the
    returned array. By default an array of the same dtype as input
    will be created.
mode : {'reflect', 'constant', 'nearest', 'mirror', 'wrap'}, optional
    The `mode` parameter determines how the input array is extended
    beyond its boundaries. Default is 'reflect'. Behavior for each valid
    value is as follows:

    'refle

In [75]:
??jensenshannon

Signature: jensenshannon(p, q, base=None, *, axis=0, keepdims=False)
Source:   
def jensenshannon(p, q, base=None, *, axis=0, keepdims=False):
    """
    Compute the Jensen-Shannon distance (metric) between
    two probability arrays. This is the square root
    of the Jensen-Shannon divergence.

    The Jensen-Shannon distance between two probability
    vectors `p` and `q` is defined as,

    .. math::

       \\sqrt{\\frac{D(p \\parallel m) + D(q \\parallel m)}{2}}

    where :math:`m` is the pointwise mean of :math:`p` and :math:`q`
    and :math:`D` is the Kullback-Leibler divergence.

    This routine will normalize `p` and `q` if they don't sum to 1.0.

    Parameters
    ----------
    p : (N,) array_like
        left probability vector
    q : (N,) array_like
        right probability vector
    base : double, optional
        the base of the logarithm used to compute the output
        if not given, then the routine uses the default base of
        scipy.stats.entropy.
    a

In [78]:
jensenshannon(pred_profile_prob,pred_profile_prob,axis=1)

array([ 0., inf, nan])

In [74]:
np.median(jensenshannon(pred_profile_prob,pred_profile_prob,axis=1))

nan